In [3]:
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
emotions = ["angry","fear","happy","neutral","sad","surprise"]
dataset_dir = "dataset/train"
#https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data

In [ ]:
#Example Data Visualization
for emotion in emotions:
    path = os.path.join(dataset_dir,emotion)
    for img in os.listdir(path):
        img_array = cv.imread(os.path.join(path,img))
        plt.imshow(cv.cvtColor(img_array,cv.COLOR_BGR2RGB))
        plt.title(emotion, fontsize=15)
        plt.show()
        break      

In [4]:
training_data = []

def create_training_data():
    for emotion in emotions:
        path = os.path.join(dataset_dir,emotion)
        class_label = emotions.index(emotion)
        for img in os.listdir(path):
            img_array = cv.imread(os.path.join(path,img))
            training_data.append([img_array,class_label])

In [5]:
create_training_data()

In [6]:
print(len(training_data))

28273


In [7]:
X = [] ##feature
y = [] ##labels

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1,48,48,3)

In [8]:
X.shape

(28273, 48, 48, 3)

In [10]:
X = X.astype(np.uint8)


In [11]:
y = np.array(y)
y = y.astype(np.uint8)
X_rescaled = X/255

In [12]:
import keras

In [15]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3),activation='relu', padding='same',input_shape=(48, 48,3)),
    keras.layers.Conv2D(32, (3,3),activation='relu', padding='same'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(64, (5,5),activation='relu',padding='same'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(128, (5,5),activation='relu',padding='same'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.BatchNormalization(),

    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(emotions), activation='softmax')
])

model.compile(optimizer='adam',
             loss = 'sparse_categorical_crossentropy',
             metrics='accuracy')

model.fit(X_rescaled,y,epochs=50)

Epoch 1/50
884/884 [==============================] - 152s 170ms/step - loss: 1.5915 - accuracy: 0.3585
Epoch 2/50
884/884 [==============================] - 143s 162ms/step - loss: 1.3191 - accuracy: 0.4775
Epoch 3/50
884/884 [==============================] - 143s 162ms/step - loss: 1.2113 - accuracy: 0.5311
Epoch 4/50
884/884 [==============================] - 141s 159ms/step - loss: 1.1252 - accuracy: 0.5660
Epoch 5/50
884/884 [==============================] - 142s 161ms/step - loss: 1.0619 - accuracy: 0.5898
Epoch 6/50
884/884 [==============================] - 143s 162ms/step - loss: 0.9934 - accuracy: 0.6241
Epoch 7/50
884/884 [==============================] - 142s 161ms/step - loss: 0.9355 - accuracy: 0.6462
Epoch 8/50
884/884 [==============================] - 142s 161ms/step - loss: 0.8796 - accuracy: 0.6693
Epoch 9/50
884/884 [==============================] - 142s 161ms/step - loss: 0.8152 - accuracy: 0.6925
Epoch 10/50
884/884 [==============================] - 142s 161m

In [16]:
model.save('face_detector_model.h5')

In [ ]:
evulation = model.evaluate(X,y)
print(evulation)

In [ ]:
plt.plot(model.history['acc'])
plt.plot(model.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(model.history['loss'])
plt.plot(model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()